In [1]:
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
from transformers import AutoTokenizer
from transformers import MarianMTModel, MarianTokenizer
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import json

2025-06-09 14:22:33.909061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749478954.191770      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749478954.257304      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Set seed.
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [3]:
SRC_LANGUAGE = 'ko'
TGT_LANGUAGE = 'en'
# src_model_checkpoint = 'distilbert-base-cased'
# tgt_model_checkpoint = "klue/bert-base"

model_name = "Helsinki-NLP/opus-mt-ko-en"

training_file_path = "/kaggle/input/sampledataset/sampleData.json"
cols = ["ko_text", "en_text"]

test_size = 0.25

BATCH_SIZE = 1
NUM_EPOCHS = 2
if(torch.cuda.is_available()):
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

In [4]:
token_transformer = MarianTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/842k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
token_transformer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-ko-en', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
# json_data = json.load(training_file_path)
with open(training_file_path, encoding = 'utf-8') as f:
    json_data = json.load(f)
# json_data
texts = [{col: item[col] for col in cols} for item in json_data["Text"]]

df = pd.DataFrame(texts)
df.head()

,ko_text,en_text
0,아내가 외출하기 전 끓여놓은 국과 반찬을 식판에 떠서 먹이기만 하면 되는데 말처럼 ...,What needs to be done is to scoop up the boile...
1,매번 잘 차려진 밥상에 숟가락만 들었던 내 모습이 부끄러워진다.,I'm ashamed of myself for holding only a spoon...
2,가끔 평일에 일찍 들어가 가족들과 밥을 먹을 때 왜 아내가 그렇게 아이들에게 고함을...,Sometimes when I got home early on weekdays an...
3,"분명 아이들 밥을 차린 시간은 5분이 채 되지 않았는데, 아침식사 시간은 한 시간이...",It has been less than 5 minutes since the chil...


In [7]:
train_data, test_data = train_test_split(df, test_size = test_size)

In [8]:
# Custom Dataset class.
class TranslationDataset(Dataset):
    def __init__(self, df, tokenizer, max_length = 128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        src = self.df[cols[0]].iloc[idx]
        tgt = self.df[cols[1]].iloc[idx]

        src_enc = self.tokenizer(src, return_tensors = "pt", padding="max_length", truncation=True, max_length = self.max_length)
        tgt_enc = self.tokenizer(tgt, return_tensors = "pt", padding="max_length", truncation=True, max_length = self.max_length)

        input_ids = src_enc["input_ids"].squeeze()
        attention_mask = src_enc["attention_mask"].squeeze()
        labels = tgt_enc["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100 #ignore padding in loss calculation
        return{
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels":labels
        }

In [9]:
train_dataset = TranslationDataset(train_data, token_transformer)
valid_dataset = TranslationDataset(test_data, token_transformer)
iterator = iter(train_dataset)
print(next(iterator))

{'input_ids': tensor([ 1937, 14270,  8189,    51,  1643,  1412, 15645,   309, 12068,  4827,
         4491, 39711,     3,     9, 39027, 15645,    54,  2660,  5795, 15856,
            9, 48114,     2,     0, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
        65000, 65000, 65000, 65000, 65000, 65000, 

In [10]:
model = MarianMTModel.from_pretrained(model_name).to(DEVICE)

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

In [11]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")
print(model)

77,943,296 total parameters.
77,943,296 training parameters.
MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=Tr

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

In [14]:
train_dataset[0]

{'input_ids': tensor([ 1937, 14270,  8189,    51,  1643,  1412, 15645,   309, 12068,  4827,
          4491, 39711,     3,     9, 39027, 15645,    54,  2660,  5795, 15856,
             9, 48114,     2,     0, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000, 65000,
         65000, 65000, 65000, 65000, 65

In [15]:
model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

src_text = ["안녕하세요. 만나서 반갑습니다."]
inputs = tokenizer(src_text, return_tensors="pt", padding=True)

inputs

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


{'input_ids': tensor([[ 4192,     2, 12299, 33891,     2,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [16]:
# train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn)
def train(model, optimizer, num_epochs, dataloader):
    print('Training')
    model.to(DEVICE)
    model.train()

    train_loss = []
    for epoch in range(num_epochs):
        epoch_loss = 0
        num_iter = 0
        for batch in dataloader:
            input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
            attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
            labels = torch.tensor(batch['labels']).to(DEVICE)
    
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            epoch_loss += loss.item()
            num_iter += 1
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(f"Epoch {epoch}, iter = {num_iter}, Loss: {loss.item()}")
        train_loss.append(epoch_loss/num_iter)
    return train_loss

def evaluate(model, dataloader):
    print('Validating')
    model.eval()
    losses = 0
    num_iter = 0
    # valid_loss = []
    for batch in dataloader:
        input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
        attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
        labels = torch.tensor(batch['labels']).to(DEVICE)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        losses += loss.item()
        num_iter += 1
        print(f"iter = {num_iter}, Loss: {loss.item()}")
    # valid_loss.append(epoch_loss/num_iter)
    return losses/num_iter

In [17]:
train(model, optimizer, NUM_EPOCHS, train_dataloader)

Training


/tmp/ipykernel_35/4112800121.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/4112800121.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/4112800121.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


Epoch 0, iter = 1, Loss: 7.429810047149658
Epoch 0, iter = 2, Loss: 6.215972423553467
Epoch 0, iter = 3, Loss: 7.482779026031494
Epoch 1, iter = 1, Loss: 7.552974224090576
Epoch 1, iter = 2, Loss: 6.381274223327637
Epoch 1, iter = 3, Loss: 7.307322978973389


[7.042853832244873, 7.0805238087972]

In [18]:
evaluate(model, valid_dataloader)

Validating
iter = 1, Loss: 6.406202793121338


/tmp/ipykernel_35/4112800121.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(batch['input_ids']).to(DEVICE)
/tmp/ipykernel_35/4112800121.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(batch['attention_mask']).to(DEVICE)
/tmp/ipykernel_35/4112800121.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['labels']).to(DEVICE)


6.406202793121338

In [19]:
#save the model
model.save_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning")

#save tokenizer
tokenizer.save_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/tokenizer_config.json',
 '/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/special_tokens_map.json',
 '/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/vocab.json',
 '/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/source.spm',
 '/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/target.spm',
 '/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning-token/added_tokens.json')

In [20]:
#load the model
finetuned_model = MarianMTModel.from_pretrained("/kaggle/working/Helsinki-NLP/opus-mt-ko-en-finetuning").to(DEVICE)


In [21]:
src_text = ["안녕하세요. 만나서 반갑습니다."]
inputs = tokenizer(src_text, return_tensors="pt", padding=True).to(DEVICE)

outputs = finetuned_model.generate(**inputs)

translation = tokenizer.decode(outputs[0], skip_special_tokens = True)

In [22]:
translation

'Hello. Nice to meet you.'